In [3]:
import pandas as pd
import numpy as np 
import json 
import requests
from bs4 import BeautifulSoup
from datetime import date

dataset = pd.read_csv(
        '../airflow/output_data/raw/shopcar/scrapping_anuncios_shopcar.csv', encoding='latin')

# geração de id do shop car
dataset.columns = ['Unnamed: 0', 'Modelo', 'Ano', 'Cor',
                    'Combustivel', 'KM', 'Preco', 'Link', 'Vendedor', 'Cidade']

dataset['Id_Anuncio_ShopCar'] = dataset['Link'].str.slice(start=-7)
dataset.reset_index()
dataset.set_index('Id_Anuncio_ShopCar', inplace=True)
dataset.drop(labels=['Unnamed: 0'], axis=1)
dataset = dataset[dataset.Preco != 'DETALHES']
# correção do campo data ano/modelo

ano_fabricacao = pd.Series(dataset['Ano']).str.slice(stop=2)
dataset['Ano_Fabricacao'] = pd.to_numeric(ano_fabricacao, errors='coerce')
dataset['Ano_Fabricacao'] = np.where(
    dataset['Ano_Fabricacao'] < 22, 2000 + dataset['Ano_Fabricacao'], 1900 + dataset['Ano_Fabricacao'])

ano_modelo = pd.Series(dataset['Ano']).str.slice(start=3, stop=5)
dataset['Ano_Modelo'] = pd.to_numeric(ano_modelo)
dataset['Ano_Modelo'] = np.where(
    dataset['Ano_Modelo'] < 22, 2000 + dataset['Ano_Modelo'], 1900 + dataset['Ano_Modelo'])
dataset = dataset.drop(labels=['Ano'], axis=1)
dataset = dataset.drop(labels=['Unnamed: 0'], axis=1)

# correção do campo KM
dataset['KM'] = dataset['KM'].str.replace('Km', '')
dataset['KM'] = dataset['KM'].str.replace('.', '')
dataset['KM'] = pd.to_numeric(dataset['KM'], errors='coerce')

# correção do campo PREÇO
dataset['Preco'] = dataset['Preco'].str.replace('R', '')
dataset['Preco'] = dataset['Preco'].str.replace('$', '')
dataset['Preco'] = dataset['Preco'].str.replace('.', '')
dataset['Preco'] = dataset['Preco'].str.replace(',00', '')
dataset['Preco'] = pd.to_numeric(dataset['Preco'], errors='ignore')

# endereços
enderecos = pd.Series(dataset['Cidade'])
dataset['Estado_Anuncio'] = enderecos.str.partition('/')[2]
dataset['Endereco_Anuncio'] = enderecos.str.partition('-')[0]

enderecos = enderecos.str.partition('-')[2]
enderecos.str.partition(' - ')
dataset['Bairro_Anuncio'] = enderecos.str.partition('-')[0]

enderecos = enderecos.str.partition('-')[2]
enderecos = enderecos.str.partition('/')
dataset['Cidade_Anuncio'] = enderecos[0]

# marca
marca = dataset['Link'].str.partition('/')[2]
dataset['Marca'] = marca.str.partition(
    '/')[2].str.partition('/')[2].str.partition('/')[2].str.partition('/')[0]
dataset['Data_Extracao_Dados'] = ''

df_anuncios = dataset
df_anuncios #.to_csv(f'../airflow/output_data/staging/anuncios_shopcar2.csv',
            #       header=True, encoding='latin')

# df_marca_modelo = pd.read_csv(
#     '../airflow/output_data/raw/scrapping_marcas_modelos_fipe.csv', encoding='latin')
# df_fuzzy = fuzzy_left_join(
#     df_anuncios, df_marca_modelo, 'Modelo', 'nome_modelo')
# df_fuzzy = df_fuzzy.drop(labels=['__id_left', '__id_right'], axis=1)
# df_fuzzy.sort_values(by='best_match_score')

C:\Users\CS342370\AppData\Local\Temp/ipykernel_123164/2678815144.py:36: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  dataset['KM'] = dataset['KM'].str.replace('.', '')
C:\Users\CS342370\AppData\Local\Temp/ipykernel_123164/2678815144.py:41: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  dataset['Preco'] = dataset['Preco'].str.replace('$', '')
C:\Users\CS342370\AppData\Local\Temp/ipykernel_123164/2678815144.py:42: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  dataset['Preco'] = dataset['Preco'].str.replace('.', '')


,Modelo,Cor,Combustivel,KM,Preco,Link,Vendedor,Cidade,Ano_Fabricacao,Ano_Modelo,Estado_Anuncio,Endereco_Anuncio,Bairro_Anuncio,Cidade_Anuncio,Marca,Data_Extracao_Dados
Id_Anuncio_ShopCar,,,,,,,,,,,,,,,,
1186860,C3 Hatch Exclusive Solaris 1.6 16v A/T,Prata,Flex,NaN,27000,https://www.shopcar.com.br/veiculos/citro-n/c3...,Monyze Gabrielle de OLiveira,Campo Grande - MS,2010.0,2011.0,,Campo Grande,MS,,citro-n,
1185801,Fox Trendline 1.0 4p.,Preta,Flex,NaN,27000,https://www.shopcar.com.br/veiculos/vw-volkswa...,Jefferson de Morais Cardoso,Campo Grande - MS,2008.0,2009.0,,Campo Grande,MS,,vw-volkswagen,
1190019,Corsa Sedan Classic LS 1.0,Prata,Flex,NaN,27500,https://www.shopcar.com.br/veiculos/gm-chevrol...,Bandi Car Veículos,"Av. Bandeirantes, 1878 - Taquarussu - Campo Gr...",2010.0,2011.0,MS,"Av. Bandeirantes, 1878",Taquarussu,Campo Grande,gm-chevrolet,
1184975,Celta Spirit 1.0 4p.,Branca,Flex,NaN,27900,https://www.shopcar.com.br/veiculos/gm-chevrol...,Bandi Car Veículos,"Av. Bandeirantes, 1878 - Taquarussu - Campo Gr...",2010.0,2011.0,MS,"Av. Bandeirantes, 1878",Taquarussu,Campo Grande,gm-chevrolet,
1189890,Fusca 1300 L,Branca,Gasolina,NaN,28000,https://www.shopcar.com.br/veiculos/vw-volkswa...,Premium Motors,"Rua João Pedro de Souza,73 - Jardim Monte Líba...",1968.0,1968.0,MS,"Rua João Pedro de Souza,73",Jardim Monte Líbano,Campo Grande,vw-volkswagen,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1232901,Vectra Elite 2.4 16v A/T,Preta,Flex,NaN,Â 26900,https://www.shopcar.com.br/veiculos/gm-chevrol...,Wyllian Teixeira Wincler,Dourados - MS,2007.0,2007.0,,Dourados,MS,,gm-chevrolet,
1220823,Linea HLX 1.9 16v A/T,Preta,Flex,NaN,Â 27500,https://www.shopcar.com.br/veiculos/fiat/linea...,Mauro Luiz,Campo Grande - MS,2010.0,2010.0,,Campo Grande,MS,,fiat,
1220332,Logan Expression 1.6 16v A/T,Preta,Flex,NaN,Â 27900,https://www.shopcar.com.br/veiculos/renault/lo...,Mauro Luiz,Campo Grande - MS,2013.0,2013.0,,Campo Grande,MS,,renault,


In [ ]:
df_shopcar